# About: Amazon S3からのリストア

---

Moodle構築環境のデータ、設定ファイルなどを Amazon S3 からリストアします。

## 概要

Amazon S3に作成したMoodle環境のバックアップからのリストアを行います。

### 前提条件


この Notebook を実行するには事前に以下のものを準備する必要があります。

* バックアップの保存先であるAmazon S3にアクセルするためのクレデンシャル
* リストア先となるVCノード/EC2インスタンス/Azure仮想マシン

リストア先となる環境は「011-VCノードの作成」、「012-EC2インスタンスの作成」、「013-Azure仮想マシンの作成」のいずれかのNotebookで作成することができます。

また、リストアの操作により既存の環境を損ねることを避けるため、リストア先となるディレクトリ `/srv/moodle` が存在していないことを前提としています。

## パラメータ設定

### リストア先のMoodle環境を指定する

リストア先となるノードをAnsibleのグループ名で指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group =

リストア対象のノードにアクセスできることを確認します。

In [ ]:
!ansible {target_group} -m ping

リストア先となるディレクトリ `/srv/moodle` が存在していないことを確認します。

In [ ]:
!ansible {target_group} -a 'test ! -e /srv/moodle'

### AWSの認証情報

Amazon S3を利用するための認証情報を指定します。

Amazonのアクセスキーを指定してください。

In [ ]:
from getpass import getpass
aws_access_key = getpass()

Amazonのシークレットキーを指定してください。

In [ ]:
aws_secret_key = getpass()

### バックアップの保存先を指定する

バックアップの保存先である Amazon S3 のバケット名、パスを指定します。

Amazon S3 のバケット名を指定してください。

In [ ]:
# (例)
# s3_bucket = 's3-bucket-name'

s3_bucket =

リストアを行うバックアップファイルを選択するために、バックアップファイルの一覧を確認します。

In [ ]:
!ansible {target_group} -a \
    'env AWS_ACCESS_KEY_ID={aws_access_key} AWS_SECRET_ACCESS_KEY={aws_secret_key} \
    aws s3 ls --recursive s3://{s3_bucket}/'

リストアを行うバックアップファイルを格納しているディレクトリを指定してください。

In [ ]:
# (例)
# backup_dir = 'moodle-simple/Moodle/2020-XX-XXTXX:XX:XX.XXXXXX'

backup_dir =

指定したバケット名、ディレクトリにバックアップファイルがあることを確認します。次のセルを実行してMoodle環境のバックアップファイル `db.sql.gz`, `moodle.tar.gz` が存在していることを確認してください。

In [ ]:
!ansible {target_group} -a \
    'env AWS_ACCESS_KEY_ID={aws_access_key} AWS_SECRET_ACCESS_KEY={aws_secret_key} \
    aws s3 ls s3://{s3_bucket}/{backup_dir}/'

## リストア

Moodle環境のリストアを行います。

### ディレクトリの作成

リストア先となるディレクトリを作成します。

In [ ]:
!ansible {target_group} -b -m file -a \
    'path=/srv/moodle state=directory owner={{{{ansible_ssh_user}}}}'

デフォルトのコンテナログの出力先ディレクトリを作成します。

In [ ]:
!ansible {target_group} -b -m file -a 'path=/var/log/httpd state=directory'
!ansible {target_group} -b -m file -a 'path=/var/log/mysql owner=999 group=adm state=directory'

### コンテナ構成、Moodleデータなどのリストア

設定ファイルなどのバックアップファイル `moodle.tar.gz` をリストアします。

リストア対象のバックアップファイルを確認します。

In [ ]:
s3_moodle_backup = f's3://{s3_bucket}/{backup_dir}/moodle.tar.gz'
print(s3_moodle_backup)

リストアを行います。

In [ ]:
!ansible {target_group} -m shell -a \
    'env AWS_ACCESS_KEY_ID={aws_access_key} AWS_SECRET_ACCESS_KEY={aws_secret_key} \
    aws s3 cp {s3_moodle_backup} - \
    | bash -c "sudo tar xzpf - -C /srv/moodle"'

リストアされたことを確認します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle tree -L 3 -F'

### DBデータ

DBデータを復元するためのSQLファイルを取得します。

リストア対象のバックアップファイルを確認します。

In [ ]:
s3_db_backup = f's3://{s3_bucket}/{backup_dir}/db.sql.gz'
print(s3_db_backup)

リストア先となるディレクトリを作成します。

In [ ]:
!ansible {target_group} -b -m file -a \
    'path=/srv/moodle/db/data state=directory owner={{{{ansible_ssh_user}}}}'
!ansible {target_group} -b -m file -a \
    'path=/srv/moodle/db/sql state=directory owner={{{{ansible_ssh_user}}}}'

DBデータをリストアするためのSQLファイルファイルを配置します。配置したSQLファイルはDBコンテナ起動時に自動的に実行されます。

In [ ]:
!ansible {target_group} -a \
    'env AWS_ACCESS_KEY_ID={aws_access_key} AWS_SECRET_ACCESS_KEY={aws_secret_key} \
    aws s3 cp {s3_db_backup} /srv/moodle/db/sql/backup.sql.gz'

### コンテナイメージ

Dockerコンテナイメージのリストアを行います。

> コンテナイメージのバックアップを作成していない、あるいはレポジトリのコンテナイメージを利用する場合はこの節を実行せずにスキップしてください。

コンテナイメージをリストアする前の状態を確認しておきます。コンテナイメージの一覧を表示します。

In [ ]:
!ansible {target_group} -a 'docker images'

リストア対象のバックアップファイルを確認します。

In [ ]:
s3_cnt_img = f's3://{s3_bucket}/{backup_dir}/container-image.tar.gz'
print(s3_cnt_img)

コンテナイメージをリストアします。

> バックアップファイルが存在していない場合はエラーになります。

In [ ]:
!ansible {target_group} -m shell -a \
    'env AWS_ACCESS_KEY_ID={aws_access_key} AWS_SECRET_ACCESS_KEY={aws_secret_key} \
    aws s3 cp {s3_cnt_img} - | gzip -cd | docker load'

リストアを行った後の状態を確認します。コンテナイメージの一覧を表示します。

In [ ]:
!ansible {target_group} -a 'docker images'

### コンテナの起動

リストアした環境のコンテナを起動します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker-compose up -d'

コンテナの状態を確認します。`State`が`Up`となっていることを確認してください。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker-compose ps'

### crontab

crontabの設定を復元します。

現在の crontab の設定を確認します。

In [ ]:
try:
    !ansible {target_group} -a 'crontab -l'
except:
    pass

バックアップファイルからcrontabの復元を行います。

In [ ]:
!ansible {target_group} -a 'crontab /srv/moodle/misc/crontab'

復元後の設定を確認します。

In [ ]:
!ansible {target_group} -a 'crontab -l'

### logrotate

コンテナログのログローテーションの設定を復元します。

In [ ]:
!ansible {target_group} -b -m shell -a \
    'cp /srv/moodle/misc/logrotate.d/* /etc/logrotate.d/'

### メンテナンスモードの解除

メンテナンスモードに変更してからバックアップを作成した場合はメンテナンスモードの解除が必要となります。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker-compose exec -T moodle \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --disable'